# Trying the model

## Update mechanism
The implementation uses semi-implicit Euler integration to update the next state based on the predicted accelerations:
$$\dot{\mathbf{p}}^{t+1}=\dot{\mathbf{p}}^t+\Delta t\cdot \ddot{\mathbf{p}}^t $$
$$\mathbf{p}^{t+1}=\mathbf{p}^t+\Delta t\cdot \dot{\mathbf{p}}^{t+1}$$
where $\Delta t=1$ for simplicity. 



In [ ]:
%load_ext autoreload
%cd ..

In [ ]:
%%writefile open_gns/simulator.py

from torch_geometric.data import Data
from torch_geometric.transforms import RadiusGraph
from open_gns.models import EncodeProcessDecode

class Simulator():
    def __init__(self, *, positions, properties, velocities=None, device=None, R=0.08):
        # initialize the model
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        checkpoint = torch.load('checkpoint_1_5.916705061520588e-06.pt')
        input_size=20
        model = EncodeProcessDecode(input_size).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        self.find_edges = RadiusGraph(R)
        self.model = model
        self.positions = positions.to(device)
        self.properties = properties.to(device)
        self.velocities = velocities if velocities is not None else torch.zeros((len(positions), 5*3))
        self.velocities = self.velocities.to(device)
        self.data = self.make_graph(positions, properties, self.velocities)

    
    def make_graph(self, positions, properties, velocities):
        x = torch.cat([positions, properties, velocities], 1)
        data = Data(x=x, pos=positions)
        data = self.find_edges(data)
        return data
    
    def step(self):
        # Predict accelerations
        data = self.data
        accelerations_ = self.model(data.x, data.edge_index)
        velocities_ = data.x[:,-3:] + accelerations_
        print(velocities_[0])
        positions_ = data.pos + velocities_
        # Reconstruct data for next frame
        self.velocities = torch.cat([self.velocities[:,3:], velocities_], 1)
        self.data = self.make_graph(positions_, self.properties, self.velocities)
        return positions_, velocities_, accelerations_

In [ ]:
# load
%autoreload 2
import torch
from open_gns.dataset import GNSDataset
from torch.nn import MSELoss
from open_gns.utils import animate_rollout

mse = MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load dataset
dataset = GNSDataset('./notebooks', split='test')
# Perform rollout using the simulator
rollout = dataset[0:143]
data = rollout[0]
data = data.to(device)
v = data.x[:,-15:]
print(v.size())
sim = Simulator(positions=data.pos, velocities=v, properties=data.x[:,3:5], device=device)
positions = []
pos_gt = []
for i, data in enumerate(rollout[1:]):
    pos_gt.append(data.pos)
    data = data.to(device)
    # Predict
    pos, vel, acc = sim.step()
    positions.append(pos.detach().cpu())
    # Compare against dataset
    loss = mse(acc,data.y)
    print(i, pos[0])
    print(i, data.pos[0])
    print(loss.item())

animate_rollout(positions)
animate_rollout(pos_gt)

In [ ]:
print(positions)